In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import catboost as cat
import xgboost as xgb
from matplotlib import pyplot as plt
%matplotlib inline


In [2]:
path = os.path.join('/', 'data', 'pik_competition', 'sev')
train_path = os.path.join(path, 'xtrain.csv')
test_path = os.path.join(path, 'xtest.csv')
y_train_path = os.path.join(path, 'ytrain.csv')


X_train_df = pd.read_csv(train_path, header=0)
# X_test_df = pd.read_csv(test_path, header=0)
y_train_df = pd.read_csv(y_train_path, header=0)

In [18]:
X_train_df.columns

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37',
       '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49',
       '50', '51', '52', '53', '54', '55', '56', '57', '58'],
      dtype='object')

In [32]:
imp = Imputer()

cols = ['1', '4', '8', '11', '12', '13', '14', '16', '19', '20', '25', '28', '33', '34', 
        '35', '37', '38', '39', '40', '41', '42', '43', '46', '49', '52', '53', '54', '55', '56',
        '57', '58']
# cols = ['1']

X_part = pd.DataFrame(imp.fit_transform(X_train_df[cols].values))

X_train_df_dropped = X_train_df.drop(columns=cols)
print(X_train_df_dropped.columns)
# impt = Imputer(strategy='most_frequent')
impt = Imputer(strategy='most_frequent')
X_part_2 = pd.DataFrame(impt.fit_transform(X_train_df_dropped.values))

X = pd.concat([X_part, X_part_2], axis=1).values

y = y_train_df.values.ravel()

Index(['2', '3', '5', '6', '7', '9', '10', '15', '17', '18', '21', '22', '23',
       '24', '26', '27', '29', '30', '31', '32', '36', '44', '45', '47', '48',
       '50', '51'],
      dtype='object')


In [33]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [34]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)


In [40]:
train_score = list()
test_score = list()

for train_index, test_index in kf.split(X):
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    
    model = cat.CatBoostClassifier(random_state=42, iterations=1000, 
                                   logging_level='Silent', learning_rate=0.1)
    model.fit(X_train, y_train)
    
    train_score.append(roc_auc_score(y_train, model.predict(X_train)))
    test_score.append(roc_auc_score(y_test, model.predict(X_test)))
    
print('train: %s' % np.array(train_score).mean())
print('test: %s' % np.array(test_score).mean())